In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'WpOI-sfBSXe9aaVkWGIQnQ',
 'name': '61929d733ddf',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-12-16T10:09:08.849001802Z',
             'build_flavor': 'default',
             'build_hash': 'd8972a71dbbd64ff17f2f4dba9ca2c3fe09fb100',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.3.2',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.2.3'}}


In [2]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [3]:
def insert_document(document):
    response = es.index(index='my_index', body=document)
    return response.body

import json
with open('data/dummy_data.json') as f:
    documents = json.load(f)

doc_id = []
for doc in documents:
    response = insert_document(doc)
    doc_id.append(response['_id'])
pprint(doc_id)

['OKtQ8JsBkibQBq1f41tQ', 'OatQ8JsBkibQBq1f41vY', 'OqtQ8JsBkibQBq1f41vk']


## Delete Operation

This is an example of a successful operation. If the `ID` exists in the document, the `delete` operation won't return any errors.

In [4]:
response = es.delete(index='my_index', id=doc_id[0])

In [5]:
pprint(response.body)

{'_id': 'OKtQ8JsBkibQBq1f41tQ',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 3,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 2,
 'result': 'deleted'}


In [6]:
try:
    response = es.delete(index='my_index', id="id")
except Exception as e:
    print(e)

NotFoundError(404, "{'_index': 'my_index', '_id': 'id', '_version': 1, 'result': 'not_found', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 4, '_primary_term': 1}")


## Get Documents by Id

In [7]:
response = es.get(index='my_index', id=doc_id[1])
response.body

{'_index': 'my_index',
 '_id': 'OatQ8JsBkibQBq1f41vY',
 '_version': 1,
 '_seq_no': 1,
 '_primary_term': 1,
 'found': True,
 '_source': {'title': 'Sample Title 2',
  'text': 'Here is another example of a document.',
  'created_on': '2024-09-24'}}

In [11]:
try:
    response = es.get(index='my_index', id=doc_id[0])
except Exception as e:
    pprint(e)

NotFoundError("{'_index': 'my_index', '_id': 'OKtQ8JsBkibQBq1f41tQ', 'found': False}", meta=ApiResponseMeta(status=404, http_version='1.1', headers={'X-elastic-product': 'Elasticsearch', 'content-type': 'application/vnd.elasticsearch+json;compatible-with=9', 'content-length': '64'}, duration=0.0026369094848632812, node=NodeConfig(scheme='http', host='localhost', port=9200, path_prefix='', headers={'user-agent': 'elasticsearch-py/9.2.1 (Python/3.12.12; elastic-transport/9.2.1)'}, connections_per_node=10, request_timeout=10.0, http_compress=False, verify_certs=True, ca_certs=None, client_cert=None, client_key=None, ssl_assert_hostname=None, ssl_assert_fingerprint=None, ssl_version=None, ssl_context=None, ssl_show_warn=True, _extras={})), body={'_index': 'my_index', '_id': 'OKtQ8JsBkibQBq1f41tQ', 'found': False})


## Count Documents

In [31]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

doc_id = []
for doc in documents:
    response = insert_document(doc)
    doc_id.append(response['_id'])
pprint(doc_id)

['Qata8JsBkibQBq1fhFsn', 'Qqta8JsBkibQBq1fhFuy', 'Q6ta8JsBkibQBq1fhFu_']


In [32]:
count = es.count(index='my_index')
count.body

{'count': 3,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [33]:
documents

[{'title': 'Sample Title 1',
  'text': 'This is the first sample document text.',
  'created_on': '2024-09-22'},
 {'title': 'Sample Title 2',
  'text': 'Here is another example of a document.',
  'created_on': '2024-09-24'},
 {'title': 'Sample Title 3',
  'text': 'The content of the third document goes here.',
  'created_on': '2024-09-24'}]

In [34]:
search_query = {
    "match": {
        "title": "Title 1"
    }
}

response = es.count(index='my_index', query=search_query)
response.body

{'count': 3,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [35]:
response = es.search(index='my_index', query={"match_all": {}})
response.body['hits']['hits']

[{'_index': 'my_index',
  '_id': 'Qata8JsBkibQBq1fhFsn',
  '_score': 1.0,
  '_source': {'title': 'Sample Title 1',
   'text': 'This is the first sample document text.',
   'created_on': '2024-09-22'}},
 {'_index': 'my_index',
  '_id': 'Qqta8JsBkibQBq1fhFuy',
  '_score': 1.0,
  '_source': {'title': 'Sample Title 2',
   'text': 'Here is another example of a document.',
   'created_on': '2024-09-24'}},
 {'_index': 'my_index',
  '_id': 'Q6ta8JsBkibQBq1fhFu_',
  '_score': 1.0,
  '_source': {'title': 'Sample Title 3',
   'text': 'The content of the third document goes here.',
   'created_on': '2024-09-24'}}]

## Exists API

In [38]:
response = es.indices.exists(index='my_index')
response.body

True

In [41]:
response = es.exists(index='my_index', id = doc_id[0])
response.body

True